In [1]:
%cd ..

/data/chanwutk/code/apperception


In [2]:
import json
import os
import pickle

import numpy as np

from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.stages.decode_frame.parallel_decode_frame import ParallelDecodeFrame
from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.detection_2d.object_type_filter import ObjectTypeFilter
from optimized_ingestion.stages.detection_3d.from_2d_and_road import From2DAndRoad as FromD2DAndRoad
from optimized_ingestion.stages.filter_car_facing_sideway import FilterCarFacingSideway
from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.detection_estimation.segment_mapping import RoadPolygonInfo
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT
from optimized_ingestion.stages.tracking_2d.tracking_2d import Tracking2D, Tracking2DResult
from optimized_ingestion.stages.tracking_3d.from_2d_and_road import From2DAndRoad as FromT2DAndRoad
from optimized_ingestion.stages.tracking_3d.tracking_3d import Tracking3DResult
from optimized_ingestion.stages.segment_trajectory import SegmentTrajectory
from optimized_ingestion.stages.segment_trajectory.construct_segment_trajectory import SegmentPoint
from optimized_ingestion.stages.segment_trajectory.from_tracking_3d import FromTracking3D

from optimized_ingestion.video import Video
from optimized_ingestion.video_skipped import VideoSkipped
from optimized_ingestion.video_compressed import VideoCompressed

/data/chanwutk/.installs/mambaforge/envs/apperception/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 2022-11-10 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Using cache found in /data/chanwutk/code/apperception/weights/ultralytics_yolov5_master


Using cuda:0


YOLOv5 🚀 2022-11-22 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
# from optimized_ingestion.cache import disable_cache
# disable_cache()

In [4]:
BOSTON_VIDEOS = [
#     "scene-0757-CAM_FRONT",
    # "scene-0103-CAM_FRONT",
    # "scene-0553-CAM_FRONT",
    # "scene-0665-CAM_FRONT",
#     "scene-0655-CAM_FRONT_RIGHT",
    "scene-0655-CAM_BACK_RIGHT",
]

NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [5]:
import torch

In [6]:
class DataclassJSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Tracking3DResult):
            return {
                "frame_idx": o.frame_idx,
                "detection_id": tuple(o.detection_id),
                "object_id": o.object_id,
                "point_from_camera": o.point_from_camera,
                "point": o.point.tolist(),
                "bbox_left": o.bbox_left,
                "bbox_top": o.bbox_top,
                "bbox_w": o.bbox_w,
                "bbox_h": o.bbox_h,
                "object_type": o.object_type,
                "timestamp": str(o.timestamp),
            }
        if isinstance(o, Tracking2DResult):
            return {
                "detection_id": tuple(o.detection_id),
                "frame_idx": o.frame_idx,
                "object_id": o.object_id,
                "bbox_left": o.bbox_left,
                "bbox_top": o.bbox_top,
                "bbox_w": o.bbox_w,
                "bbox_h": o.bbox_h,
                "object_type": o.object_type,
                "confidence": o.confidence
            }
        if isinstance(o, SegmentPoint):
            return {
                "detection_id": tuple(o.detection_id),
                "car_loc3d": o.car_loc3d,
                "timestamp": str(o.timestamp),
                "segment_line": str(o.segment_line),
                "segment_heading": o.segment_heading,
                "road_polygon_info": o.road_polygon_info,
                "obj_id": o.obj_id,
                "next": None if o.next is None else tuple(o.next.detection_id),
                "prev": None if o.prev is None else tuple(o.prev.detection_id),
                
            }
        if isinstance(o, RoadPolygonInfo):
            return {
                "id": o.id,
                "polygon": str(o.polygon),
                "segment_lines": [str(l) for l in o.segment_lines],
                "road_type": o.road_type,
                "segment_headings": o.segment_headings,
                "contains_ego": o.contains_ego,
                "ego_config": o.ego_config,
                "fov_lines": o.fov_lines
            }
        if isinstance(o, torch.Tensor):
            return o.tolist()
        if isinstance(o, np.ndarray):
            return o.tolist()
        return super().default(o)

In [7]:
print(NUSCENES_PROCESSED_DATA in os.environ)
print(os.environ['NUSCENES_PROCESSED_DATA'])

True
/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini


In [8]:
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames-compressed.pickle"), "rb") as f:
    videos = pickle.load(f)

In [9]:
def run_benchmark(pipeline, filename):
    metadata_strongsort = {}
    metadata_3d = {}
    metadata_segment = {}
    failed_videos = []
    for name, video in videos.items():
#         try:
        print(name, '--------------------------------------------------------------------------------')
        frames = VideoCompressed(
            os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
            [camera_config(*f, 0) for f in video["frames"]],
        )

        output = pipeline.run(Payload(frames))
        metadata_strongsort[name] = StrongSORT.get(output)
#         except:
#             failed_videos.append(name)
#             print('error')
#         break

    with open(f"./outputs/{filename}.json", "w") as f:
        json.dump(metadata_strongsort, f, cls=DataclassJSONEncoder, indent=2)

    
    with open(f"./outputs/{filename}-failed-videos.json", "w") as f:
        json.dump(failed_videos, f, indent=2)

    with open(f"./outputs/{filename}-performance.json", "w") as f:
        performance = [
            {
                "stage": stage.classname(),
                "runtimes": stage.runtimes,
            }
            for stage
            in pipeline.stages
        ]
        json.dump(performance, f, indent=2)

In [10]:
type_filter = ['motorcycle', 'car', 'truck', 'bus']

In [11]:
pipeline1 = Pipeline()
pipeline1.add_filter(ParallelDecodeFrame())
# pipeline1.add_filter(DecodeFrame())
pipeline1.add_filter(filter=YoloDetection())
# pipeline1.add_filter(filter=ObjectTypeFilter(type_filter))

# pipeline1.add_filter(filter=FromD2DAndRoad())
pipeline1.add_filter(filter=StrongSORT(cache=True))
pipeline1.add_filter(filter=StrongSORT(cache=False))
# pipeline1.add_filter(filter=FromT2DAndRoad())

# pipeline1.add_filter(filter=FromTracking3D())

run_benchmark(pipeline1, 'segment-tracking-without-de')

YOLOv5 🚀 2022-11-10 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Using cache found in /data/chanwutk/code/weights/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-22 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0103-CAM_FRONT_LEFT --------------------------------------------------------------------------------
Stage:  DecodeFrame.ParallelDecodeFrame


YOLOv5 🚀 2022-11-10 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)



None
231
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection
None
231
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking2D.StrongSORT
Successfully loaded pretrained weights from "/data/chanwutk/code/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 231/231 [00:44<00:00,  5.18it/s]
YOLOv5 🚀 2022-11-10 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)



...

None
238
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.YoloDetection
None
238
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking2D.StrongSORT
Successfully loaded pretrained weights from "/data/chanwutk/code/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 238/238 [00:13<00:00, 17.29it/s]
YOLOv5 🚀 2022-11-10 Python-3.10.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN Xp, 12196MiB)



None
238
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Tracking2D.StrongSORT
Successfully loaded pretrained weights from "/data/chanwutk/code/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 238/238 [00:16<00:00, 14.84it/s]


None
238
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK


In [12]:
pipeline1.stages[2].ss_benchmarks

[[[0.0015704631805419922,
   5.0067901611328125e-06,
   0.15283703804016113,
   1.430511474609375e-06],
  [0.0024404525756835938,
   0.00849771499633789,
   0.22387099266052246,
   9.5367431640625e-07],
  [0.0009200572967529297,
   0.006749391555786133,
   0.3643040657043457,
   0.00015735626220703125],
  [0.0011267662048339844,
   0.00787663459777832,
   0.4272584915161133,
   0.00018930435180664062],
  [0.0008947849273681641,
   0.007241010665893555,
   0.379148006439209,
   0.00016260147094726562],
  [0.001176595687866211,
   0.014944076538085938,
   0.11460137367248535,
   0.00012159347534179688],
  [0.0008821487426757812,
   0.008730173110961914,
   0.1258859634399414,
   0.0001323223114013672],
  [0.0011556148529052734,
   0.0082550048828125,
   0.12773656845092773,
   0.00014925003051757812],
  [0.0008563995361328125,
   0.005484580993652344,
   0.29751157760620117,
   0.0001659393310546875],
  [0.0011365413665771484,
   0.008418798446655273,
   0.6338760852813721,
   0.00014472

In [14]:
pipeline1.stages[3].ss_benchmarks

[[[0.004833698272705078,
   6.4373016357421875e-06,
   0.020119905471801758,
   1.1920928955078125e-06],
  [0.0047032833099365234,
   0.014139175415039062,
   0.021973371505737305,
   9.5367431640625e-07],
  [0.004555940628051758,
   0.021389245986938477,
   0.024807453155517578,
   8.296966552734375e-05],
  [0.0047245025634765625,
   0.04167914390563965,
   0.026359081268310547,
   9.822845458984375e-05],
  [0.006606578826904297,
   0.0506746768951416,
   0.028181076049804688,
   0.0001423358917236328],
  [0.006724357604980469,
   0.05521512031555176,
   0.04031872749328613,
   0.00013518333435058594],
  [0.00679326057434082,
   0.05385947227478027,
   0.03225517272949219,
   0.00017309188842773438],
  [0.010179758071899414,
   0.1339740753173828,
   0.06943607330322266,
   0.000202178955078125],
  [0.009926557540893555,
   0.08629870414733887,
   0.21765542030334473,
   0.0002465248107910156],
  [0.006327152252197266,
   0.14884495735168457,
   0.6380674839019775,
   0.00021576881408

In [13]:
with open(f"./outputs/segment-tracking-without-de-performance.json", "r") as f:
    benchmark = json.load(f)

ss_cache, ss_no_cache, *_ = benchmark[4:]
benchmark_data = [
    {
        'name': ssc['name'],
        'runtime_cache': ssc['runtime'],
        'runtime_no_cache': ssnc['runtime']
    }
    for ssc, ssnc
    in zip(ss_cache['runtimes'], ss_no_cache['runtimes'])
]
benchmark_data

ValueError: not enough values to unpack (expected at least 2, got 0)

In [ ]:
import altair as alt
import pandas as pd

In [ ]:
(alt.Chart(pd.DataFrame.from_dict(benchmark_data))
    .mark_point()
    .encode(
        x='runtime_cache:Q',
        y='runtime_no_cache:Q',
        tooltip=['name']
    )
)